# LangChain: Q&A over Documents

An example might be a tool that would allow you to query a product catalog for items of interest.

In [ ]:
#pip install --upgrade langchain

In [39]:
# !pip install "langchain[docarray]"

# error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
#   ----------------------------------------
#   ERROR: Failed building wheel for hnswlib
# ERROR: Could not build wheels for hnswlib which use PEP 517 and cannot be installed directly

In [6]:
import os
os.environ['OPENAI_API_KEY']=''

In [ ]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [30]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown

In [32]:
file = 'OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file, encoding='utf8')

In [3]:
from langchain.indexes import VectorstoreIndexCreator

In [5]:
#pip install docarray
from langchain.callbacks import get_openai_callback

In [40]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch
).from_loaders([loader])

ImportError: Could not import docarray python package. Please install it with `pip install "langchain[docarray]"`.

In [19]:
# with get_openai_callback() as cb:
#     index = VectorstoreIndexCreator(
#     vectorstore_cls=DocArrayInMemorySearch
# ).from_loaders([loader])
#     print(cb)

In [ ]:
query ="Please list all your shirts with sun protection \
in a table in markdown and summarize each one."

In [ ]:
response = index.query(query)

In [ ]:
display(Markdown(response))

In [26]:
loader = CSVLoader(file_path=file, encoding='utf8')

In [24]:
# CSVLoader?

In [27]:
docs = loader.load()

In [28]:
docs[0]

Document(page_content=": 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \n\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \n\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \n\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \n\nQuestions? Please contact us for any inquiries.", metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 0})

In [15]:
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [16]:
with get_openai_callback() as cb:
    embed = embeddings.embed_query("Hi my name is Harrison")
    print(cb)


Tokens Used: 0
	Prompt Tokens: 0
	Completion Tokens: 0
Successful Requests: 0
Total Cost (USD): $0.0


In [17]:
print(len(embed))

1536


In [18]:
print(embed[:5])

[-0.021913960576057434, 0.006774206645786762, -0.018190348520874977, -0.039148248732089996, -0.014089343138039112]


In [31]:
with get_openai_callback() as cb:
    db = DocArrayInMemorySearch.from_documents(
    docs, 
    embeddings
)
    print(cb)
    


ImportError: Could not import docarray python package. Please install it with `pip install "langchain[docarray]"`.

In [ ]:
query = "Please suggest a shirt with sunblocking"

In [ ]:
docs = db.similarity_search(query)

In [ ]:
len(docs)

In [ ]:
docs[0]

In [ ]:
retriever = db.as_retriever()

In [ ]:
llm = ChatOpenAI(temperature = 0.0)


In [ ]:
qdocs = "".join([docs[i].page_content for i in range(len(docs))])


In [ ]:
response = llm.call_as_llm(f"{qdocs} Question: Please list all your \
shirts with sun protection in a table in markdown and summarize each one.") 


In [ ]:
display(Markdown(response))

In [ ]:
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

In [ ]:
query =  "Please list all your shirts with sun protection in a table \
in markdown and summarize each one."

In [ ]:
response = qa_stuff.run(query)

In [ ]:
display(Markdown(response))

In [ ]:
response = index.query(query, llm=llm)

In [ ]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=embeddings,
).from_loaders([loader])